In [1]:
%load_ext autoreload
%autoreload 2
from longeval.spark import get_spark
from longeval.collection import ParquetCollection
from pathlib import Path
from longeval.evaluation import prepare_queries, run_search, score_search

spark = get_spark()
root = Path("~/").expanduser() / "scratch/longeval"
train_en_collection = ParquetCollection(spark, f"{root}/parquet/train/2023_01/English")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/16 05:55:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/16 05:55:26 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
! curl -s localhost:9200/_cat/indices | grep longeval | sort

IOStream.flush timed out
green open longeval-test-english-2023_06  nNt2rlNoQrOE5zv1g7k0DQ 1 0 1790028 0 6.5gb 6.5gb
green open longeval-test-english-2023_08  C7IUKwb5Q3mOQuQkIgemaA 1 0 2531614 0 6.4gb 6.4gb
green open longeval-test-french-2023_06   bB-h1G1LRaC4baYUYskaVQ 1 0 1790028 0 6.9gb 6.9gb
green open longeval-test-french-2023_08   fXk4gWNTRMyHJjRDvkI9Mg 1 0 2531614 0 6.8gb 6.8gb
green open longeval-train-english-2023_01 xdfAKea2ShKJIQBJjpc_6w 1 0 2049729 0 9.1gb 9.1gb
green open longeval-train-french-2023_01  uLkqlDp2SjewGftZzpsBJA 1 0 2049729 0 9.6gb 9.6gb


In [4]:
queries = prepare_queries(train_en_collection)
queries.printSchema()
queries.show(n=3, truncate=100)

root
 |-- qid: string (nullable = true)
 |-- query: string (nullable = true)
 |-- qrel: array (nullable = false)
 |    |-- element: struct (containsNull = false)
 |    |    |-- docid: string (nullable = true)
 |    |    |-- rel: integer (nullable = true)

+-----------------+-------------------+----------------------------------------------------------------------------------------------------+
|              qid|              query|                                                                                                qrel|
+-----------------+-------------------+----------------------------------------------------------------------------------------------------+
|q0123103079215124|bill of sale vessel|[{doc012300710752, 1}, {doc012304200443, 0}, {doc012311608205, 0}, {doc012311713469, 1}, {doc0123...|
|q0123103079215188|   areches beaufort|[{doc012310509318, 1}, {doc012304303918, 2}, {doc012312404802, 0}, {doc012304816793, 1}, {doc0123...|
|q0123103079215846|     schengen space|

In [21]:
search_df = run_search(queries, "longeval-train-english-2023_01", k=1000)
search_df.printSchema()
eval_en_202301 = score_search(search_df)
eval_en_202301.describe().show()

INFO:opensearch:POST http://localhost:9200/_msearch [status:200 request:26.127s]
25/03/16 06:07:56 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


root
 |-- qid: string (nullable = true)
 |-- total: long (nullable = true)
 |-- max_score: double (nullable = true)
 |-- docids: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- scores: array (nullable = true)
 |    |-- element: double (containsNull = false)
 |-- qrel: array (nullable = false)
 |    |-- element: struct (containsNull = false)
 |    |    |-- docid: string (nullable = true)
 |    |    |-- rel: integer (nullable = true)



25/03/16 06:07:58 WARN TaskSetManager: Stage 185 contains a task of very large size (8874 KiB). The maximum recommended task size is 1000 KiB.
25/03/16 06:08:27 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-------+-------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------------+
|summary|                map|              ndcg|       ndcg_cut_10|       ndcg_cut_100|     ndcg_cut_1000|        ndcg_cut_15|        ndcg_cut_20|       ndcg_cut_200|        ndcg_cut_30|         ndcg_cut_5|       ndcg_cut_500|           ndcg_rel|              qid|
+-------+-------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------------+
|  count|                597|               597|               597|                597|               597|                597|                597|                597|                597|                597

In [25]:
# english scores
en_scores = {}
train_en_collection = ParquetCollection(spark, f"{root}/parquet/train/2023_01/English")
en_queries = prepare_queries(train_en_collection).cache()

indices = [
    "longeval-train-english-2023_01",
    "longeval-test-english-2023_06",
    "longeval-test-english-2023_08",
]
for date in ["2023_01", "2023_06", "2023_08"]:
    index = [x for x in indices if date in x][0]
    search_df = run_search(en_queries, index, k=100)
    eval_df = score_search(search_df)
    en_scores[date] = eval_df

25/03/16 06:16:38 WARN CacheManager: Asked to cache already cached data.
INFO:opensearch:POST http://localhost:9200/_msearch [status:200 request:3.637s]
25/03/16 06:16:42 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/03/16 06:16:45 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
INFO:opensearch:POST http://localhost:9200/_msearch [status:200 request:3.781s]
25/03/16 06:16:49 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/03/16 06:16:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
INFO:opensearch:POST http://localhost:9200/_msearch [status:200 request:6.163s]
25/03/16 06:16:59 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/03/16 06:17:02 WARN SparkSession: Using an existing Spark session; only runtime SQL 

In [30]:
# get means of the scores
from pyspark.sql import functions as F

scores = [
    en_scores[date].select(F.mean("ndcg_cut_10").alias("score")).first().score
    for date in ["2023_01", "2023_06", "2023_08"]
]
scores

[0.1920590306618126, 0.0, 0.0]

In [ ]:
# english scores
fr_scores = {}
train_fr_collection = ParquetCollection(spark, f"{root}/parquet/train/2023_01/French")
queries = prepare_queries(train_en_collection).cache()

indices = [
    "longeval-train-french-2023_01",
    "longeval-test-french-2023_06",
    "longeval-test-french-2023_8",
]
for date in ["2023_01", "2023_06", "2023_08"]:
    index = [x for x in indices if date in x][0]
    search_df = run_search(queries, index, k=100)
    eval_df = score_search(search_df)
    en_scores[date] = eval_df